Note: Use this template to develop your project. Do not change the steps. For each step, you may add additional cells if needed.

#### Group Information

Group No: 

- Member 1:
- Member 2:
- Member 3:
- Member 4:

#### Import Libraries

In [1]:
%config Completer.use_jedi=False
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # only shows error messages

# import necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd


#### Load the dataset

In [14]:
df = pd.read_csv('classification_dataset.csv')
df
# "f1", "f2", "f3", "f4" and "f5" determines "label" 


,f1,f2,f3,f4,f5,label
0,1.286233,15.643743,-1.879915,-11.294839,15.245472,0
1,2.853398,0.129878,17.620669,3.945204,8.157459,1
2,3.285310,3.176560,12.610554,-6.063613,1.831887,0
3,2.019516,-1.967793,9.306435,-0.938714,-1.203038,0
4,-2.326527,3.453234,13.855478,-5.236421,1.547216,0
...,...,...,...,...,...,...
995,-2.248262,-4.619586,3.248760,9.114543,4.370790,1
996,7.882330,1.942559,13.304597,-2.682707,0.623444,0
997,14.421812,-10.688891,5.242771,-2.954794,11.689658,1
998,5.566459,-4.118762,3.670333,7.948329,10.940144,1


#### Define the loss function

In [3]:
def loss_fn(y_true, y_pred):
    return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_true, y_pred))

#### Define function to perform prediction

In [17]:
def sigmoid(x):
    # returns S(x) = 1/(1+e^(-x))
    return 1 / (1 + tf.exp(-x)) 

def relu(x):
    # returns (x > y ? x : y)
    return tf.maximum(0, x)

def forward(x, weights, biases):
    """
    Forward propagation function for a two-layer fully connected neural network

    Arguments:
        x (tf.Tensor): Input tensor
        weights (list): List of weight tensors
        biases (list): List of bias tensors

    Returns:
        tf.Tensor: Output tensor.
    """
    layer1 = relu(tf.matmul(x, weights) + biases[0])
    output = tf.matmul(layer1, weights) + biases[1]
    return output

#### Define function for model training
Display the training and validation loss values for each epoch of the training loop. The displayed value must be in 6 decimal places.<br>
Hint: <br>
Use `tf.GradientTape` to compute the gradients.

In [18]:
def train(x, y, weights, biases, learning_rate):
    """ 
    compute the gradient and update the weights and biases.
    """
    with tf.GradientTape() as tape:
        y_pred = forward(x, weights, biases)
        loss = loss_fn(y, y_pred)
    gradients = tape.gradient(loss, weights + biases)
    for i in range(len(weights)):
        weights[i].assign_sub(learning_rate * gradients[i])
        biases[i].assign_sub(learning_rate * gradients[i+len(weights)])
    return loss

def fit(x_train, y_train, x_val, y_val, weights, biases, learning_rate, epochs):
    """ 
    This function implements the training loop.
    """
    train_loss_results = []
    val_loss_results = []
    for epoch in range(epochs):
        train_loss = train(x_train, y_train, weights, biases, learning_rate)
        train_loss_results.append(train_loss)
        val_loss = loss_fn(y_val, forward(x_val, weights, biases))
        val_loss_results.append(val_loss)
        print(f"Epoch {epoch}: Training loss: {train_loss}, Validation loss: {val_loss}")
    return train_loss_results, val_loss_results

#### Define the tensors to hold the weights and biases (create the model)
Hint: <br>
Use `tf.Variable` to create the tensors.<br>
Put the tensors in a list.

In [34]:
input_size = 5  # 5 features
hidden_size = 3  # Size of the hidden layer
output_size = 2  # Predict 2 classes

# Initialize the weights and biases
weights = [
    tf.Variable(tf.random.normal([input_size, hidden_size]), name='w1'),
    tf.Variable(tf.random.normal([hidden_size, output_size]), name='w2'),
]

biases = [
    tf.Variable(tf.zeros([hidden_size]), name='b1'),
    tf.Variable(tf.zeros([output_size]), name='b2'),
]

#### Split the dataset
The ratio of training and test is 7:1:2.

In [31]:
test_ratio = 0.3
validation_ratio = 1/3

'''
Split dataset into training and test with ratio of 7:3, then split the test set into test and validation with ratio of 1:3
'''

# Split the dataset into training and validation sets
train_data, test_data = train_test_split(df, test_size = test_ratio, random_state=42)

# Split the training and validation data into training and validation sets
test_data, val_data = train_test_split(test_data, test_size= validation_ratio, random_state=42)

# Print the sizes of the three sets
print("Training set size:", len(train_data))
print("Validation set size:", len(val_data))
print("Test set size:", len(test_data))


Training set size: 700
Validation set size: 100
Test set size: 200


#### Normalize the data

In [32]:
from sklearn.preprocessing import MinMaxScaler
# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Fit the scaler to the data and transform the data
df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

# I did not use sigmoid because some values are squashed into 1 although their values were different previously 
'''
df[['f1', 'f2', 'f3', 'f4', 'f5']] = sigmoid(df[['f1', 'f2', 'f3', 'f4', 'f5']]) 
print(df)
'''


"\ndf[['f1', 'f2', 'f3', 'f4', 'f5']] = sigmoid(df[['f1', 'f2', 'f3', 'f4', 'f5']]) \nprint(df)\n"

#### Train the model

In [33]:
# Define the input features and target variable
X_train = train_data.drop('label', axis=1)
y_train = train_data['label']

# Train the model
train_loss_results, val_loss_results = fit(X_train, y_train, val_data.drop('label', axis=1), val_data['label'], weights, biases, learning_rate=0.01, epochs=100)


InvalidArgumentError: {{function_node __wrapped__Pack_N_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Shapes of all inputs must match: values[0].shape = [3,5] != values[1].shape = [3,2] [Op:Pack] name: b

#### Display the training loss and validation loss against epoch graph

#### Predict the test set

#### Display the confusion matrix and the classification report.